In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import chain

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import tqdm

import copy

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support, confusion_matrix,
                             precision_score, recall_score)

import torch.utils.data as data_utils

In [ ]:
device = torch.device("cuda:0")

## Read in data 

In [ ]:
train = np.load('/content/baseline_train_smote.npz')
#train = np.load("/content/baseline_train.npz")

In [ ]:
train["x"].shape

In [ ]:
train["y"].shape

In [ ]:
#validation = np.load('/content/baseline_validation_smote.npz')
validation = np.load('/content/baseline_validation.npz')

In [ ]:
validation["x"].shape

In [ ]:
validation["y"].shape

In [ ]:
test = np.load("/content/baseline_test.npz")

In [ ]:
#test["x"].shape 

In [ ]:
#test["y"].shape

In [ ]:
unique_labels, label_counts = np.unique(train["y"], return_counts=True)
print(f"Count of negative vs positive labels: {label_counts}")

In [ ]:
231/755844

In [ ]:
class_weights = [1/c for c in label_counts]
class_weights 

In [ ]:
# Assign weights to samples 
sample_weights = [class_weights[int(l)] for l in train["y"]]
sampler = WeightedRandomSampler(sample_weights, len(train["y"]))

In [ ]:
# Load data 
minibatch_size = 500

X_train = torch.tensor(train["x"], dtype=torch.float32).to(device)
y_train = torch.tensor(train["y"], dtype=torch.float32).reshape(-1,1).to(device)
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=minibatch_size, 
                                           shuffle=True) 

X_val = torch.tensor(validation["x"], dtype=torch.float32).to(device)
y_val = torch.tensor(validation["y"], dtype=torch.float32).reshape(-1,1).to(device)
val_dataset = torch.utils.data.TensorDataset(X_val, y_val)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False)

X_test = torch.tensor(test["x"], dtype=torch.float32).to(device)
y_test = torch.tensor(test["y"], dtype=torch.float32).reshape(-1,1).to(device)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

## Model setup 

In [ ]:
class FF1(nn.Module):
    def __init__(self, input_size, hl1, hl2):
        super().__init__()
        self.hl1 = nn.Linear(input_size, hl1)
        self.hl2 = nn.Linear(hl1, hl2)
        self.output = nn.Linear(hl2, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.d1 = nn.Dropout(p=0.7)
        self.d2 = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.relu(self.hl1(x))
        x = self.d1(x) # added
        x = self.relu(self.hl2(x))
        x = self.d2(x) # added 
        x = self.sigmoid(self.output(x))

        return x

In [ ]:
class FF1_best(nn.Module):
    def __init__(self):
        super().__init__()
        self.hl1 = nn.Linear(input_size, 24)
        self.hl2 = nn.Linear(24, 29)
        self.hl3 = nn.Linear(29, 51)
        self.output = nn.Linear(51, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.d1 = nn.Dropout(p=0.624642706940751)
        self.d2 = nn.Dropout(p=0.5871473304012731)
        self.d3 = nn.Dropout(p=0.5806565872839119)

    def forward(self, x):
        x = self.relu(self.hl1(x))
        x = self.d1(x) # added
        x = self.relu(self.hl2(x))
        x = self.d2(x) # added 
        x = self.relu(self.hl3(x))
        x = self.d3(x)
        x = self.sigmoid(self.output(x))

        return x

In [ ]:
class FF2(nn.Module):
    def __init__(self, input_size, hl1, hl2, hl3, hl4, hl5):
        super().__init__()
        self.hl1 = nn.Linear(input_size, hl1)
        self.hl2 = nn.Linear(hl1, hl2)
        self.hl3 = nn.Linear(hl2, hl3)
        self.hl4 = nn.Linear(hl3, hl4)
        self.hl5 = nn.Linear(hl4, hl5)
        self.output = nn.Linear(hl5, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.hl1(x))
        x = self.relu(self.hl2(x))
        x = self.relu(self.hl3(x))
        x = self.relu(self.hl4(x))
        x = self.relu(self.hl5(x))
        x = self.sigmoid(self.output(x))

        return x

In [ ]:
# Initialize model 
#input_size = 106
#hl1 = 66 
#hl2 = 44
#hl1 = 100 
#hl2 = 80
#hl3 = 80
#hl4 = 60
#hl5 = 40

# prediction threshold
pred_threshold_low = 0.0003
pred_threshold_normal = 0.5

curr_model = FF1_best()
#curr_model = FF1(input_size, hl1, hl2)
#curr_model = FF2(input_size, hl1, hl2, hl3, hl4, hl5)
curr_model = curr_model.to(device)

# Initialize Hyperparameters
#learning_rate = 0.001
learning_rate = 0.00028934840305875725
num_epochs = 200
criterion = nn.BCELoss()
#criterion = nn.BCEWithLogitsLoss()
#optimizer = torch.optim.Adam(curr_model.parameters(), 
#                             lr=learning_rate, 
#                             weight_decay=10e-05)
optimizer = torch.optim.RMSprop(curr_model.parameters(), 
                             lr=learning_rate, 
                             weight_decay=10e-05)

In [ ]:
print(sum([x.reshape(-1).shape[0] for x in curr_model.parameters()])) #number of parameters 

In [ ]:
train_results = {}
train_results["train_loss"] = []
train_results["epoch"] = []
train_results["train_pred"] = []
train_results["train_label"] = []
train_results["train_pred_label_0pt0003"] = []
train_results["train_pred_label_0pt5"] = []

train_metrics = {}
train_metrics["f1_fraud_0pt0003"] = []
train_metrics["f1_fraud_0pt5"] = []

for epoch in range(num_epochs):
  h = np.array([])
  for data, label in train_loader:
    curr_f1_scores = []

    # move data and label to device
    data = data.to(device)
    label = label.to(device)
    
    # forward
    output = curr_model(data)
    loss = criterion(output, label)
    h = np.append(h, loss.item())

    # backward 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  #mean_f1 = np.mean(curr_f1_scores)
  #print(f"Epoch {epoch} fraud mean F1 score at 0.5 threshold: {mean_f1}")
  #train_metrics["f1_fraud_0pt5"].append(mean_f1)

  # Record loss values 
  mean_loss = np.mean(h)
  print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, mean_loss))
  #train_results['train_loss'].append(mean_loss)

In [ ]:
output

In [ ]:
train_summary_metrics = {}
train_summary_metrics["train_pred"] = []
train_summary_metrics["train_label"] = []

curr_model.eval()

with torch.no_grad():
  train_output = curr_model(X_train)
  train_output = list(chain.from_iterable(train_output.tolist()))

  train_summary_metrics["train_pred"] = train_output
  train_summary_metrics["train_label"] = list(chain.from_iterable(y_train.tolist()))

In [ ]:
train_summary_metrics_df = pd.DataFrame(train_summary_metrics)

In [ ]:
f1_score(train_summary_metrics_df.train_label, 
                                 train_summary_metrics_df.train_pred.round())

In [ ]:
#torch.save(curr_model.state_dict(), "/content/smote_model_3_layers_initial_dropout_100_epoch")

In [ ]:
# Store results for plotting 
validation_results = {}
validation_results['validation_loss'] = []
validation_results['validation_pred'] = []
validation_results['validation_true_label'] = []

# To store accuracy
accuracy = {}
accuracy['validation_accuracy'] = []
accuracy['test_accuracy'] = []

# Evaluate accuracy on validation data 
curr_model.eval()

with torch.no_grad():
    for data, label in val_loader:
        # move data and label to device
        data = data.to(device)
        label = label.to(device)

        output = curr_model(data).to(device)
        #print(f"Output: {output}")

        # store relevant values 
        curr_loss = criterion(output, label).item()
        curr_label = label.item()
        curr_output = output.item()
        #print(f"Loss: {loss}")
        validation_results['validation_loss'].append(curr_loss)
        validation_results['validation_true_label'].append(curr_label)
        validation_results['validation_pred'].append(curr_output)

        # accuracy
        acc = (output.round() == label).float().mean()
        acc = float(acc)
        #print(f"Accuracy: {acc}")
        accuracy["validation_accuracy"].append(acc) 

In [ ]:
validation_results_df = pd.DataFrame(validation_results)

In [ ]:
validation_results_df.head()

In [ ]:
#validation_results_df.to_csv("/content/validation_results_df_SMOTE_100epoch_500batch.csv.csv", index=False)

In [ ]:
fpr, tpr, thresholds = roc_curve(validation_results_df.validation_true_label, 
                                 validation_results_df.validation_pred)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.001, 1])
plt.ylim([0, 1.001])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show();

In [ ]:
precision, recall, thresholds = precision_recall_curve(validation_results_df.validation_true_label, 
                                 validation_results_df.validation_pred)

plt.plot(recall, precision, 'b', label='Precision-Recall curve')
plt.title('Recall vs Precision')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

In [ ]:
class_names = ["Normal", "Fraud"]
cr = classification_report(validation_results_df.validation_true_label, 
                                 validation_results_df.validation_pred.round(),
                      target_names=class_names,
                      output_dict=True) 
print(cr)

In [ ]:
cr["Normal"]

In [ ]:
cr["Fraud"]

In [ ]:
confusion_matrix(validation_results_df.validation_true_label, 
                                 validation_results_df.validation_pred.round())

In [ ]:
f1_score(validation_results_df.validation_true_label, 
                                 validation_results_df.validation_pred.round())

In [ ]:
precision_score(validation_results_df.validation_true_label, 
                                 validation_results_df.validation_pred.round())

In [ ]:
recall_score(validation_results_df.validation_true_label, 
                                 validation_results_df.validation_pred.round())

In [ ]:
sum(accuracy["validation_accuracy"])/len(accuracy["validation_accuracy"])

## Test data

In [ ]:
# Store results for plotting 
test_results = {}
test_results['test_loss'] = []
test_results['test_pred'] = []
test_results['test_true_label'] = []

# Evaluate accuracy on validation data 
curr_model.eval()

with torch.no_grad():
    for data, label in test_loader:
        # move data and label to device
        data = data.to(device)
        label = label.to(device)

        output = curr_model(data).to(device)
        #print(f"Output: {output}")

        # store relevant values 
        curr_loss = criterion(output, label).item()
        curr_label = label.item()
        curr_output = output.item()
        print(f"Loss: {loss}")
        test_results['test_loss'].append(curr_loss)
        test_results['test_true_label'].append(curr_label)
        test_results['test_pred'].append(curr_output)

        # accuracy
        acc = (output.round() == label).float().mean()
        acc = float(acc)
        #print(f"Accuracy: {acc}")
        accuracy["validation_accuracy"].append(acc) 

In [ ]:
test_results_df = pd.DataFrame(test_results)

In [ ]:
sum(accuracy["validation_accuracy"])/len(accuracy["validation_accuracy"])

In [ ]:
f1_score(test_results_df.test_true_label, 
                                 test_results_df.test_pred.round())

In [ ]:
fpr, tpr, thresholds = roc_curve(test_results_df.test_true_label, 
                                 test_results_df.test_pred)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.001, 1])
plt.ylim([0, 1.001])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show();

In [ ]:
precision, recall, thresholds = precision_recall_curve(test_results_df.test_true_label, 
                                 test_results_df.test_pred)

plt.plot(recall, precision, 'b', label='Precision-Recall curve')
plt.title('Recall vs Precision')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

In [ ]:
class_names = ["Normal", "Fraud"]
cr = classification_report(test_results_df.test_true_label, 
                                 test_results_df.test_pred.round(),
                      target_names=class_names,
                      output_dict=True) 

In [ ]:
cr["Normal"]

In [ ]:
cr["Fraud"]

In [ ]:
confusion_matrix(test_results_df.test_true_label, 
                                 test_results_df.test_pred.round())